Ironhack_gambling.ipynb


In [2]:
import pymysql

cnx = pymysql.connect(user='root', password='Malcomx1',
                      host='localhost', database='sakila')
if cnx.open:
    print("Connection open")
else:
    print("Connection is not successfully open")

Connection open


In [3]:
# the object which will interact with the database
cursor = cnx.cursor()